<a href="https://colab.research.google.com/github/rickqiu/trax_chatbots/blob/main/reformer_chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License")
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

 https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reformer: Chatbots
@author: Rick Qiu

This notebook demonstrates examples of conversations between two chatbots, who have local knowledge in multiple domains such as attraction, hospital, hotel, police, taxi and train. 

The demo uses Google Trax deep learning library and a pre-trained Reformer language model "chatbot_model1.pkl.gz" from COURSERA NLP Specialization. 

Here is a list of useful resources.


- [MultiWoz](https://arxiv.org/abs/1810.00278) dataset
- [Reformer](https://arxiv.org/abs/2001.04451) paper
- [Trax](https://github.com/google/trax) code repository
- COURSERA [NLP Specialization](https://www.coursera.org/specializations/natural-language-processing)

### Acknowledgements
Many thanks to Łukasz Kaiser and the Google Trax open source project team members

Special thanks to NLP Specialization instructor Younes Bensouda Mourri and all staff






## 1. Setup

In [ ]:
!pip install -q -U https://storage.googleapis.com/jax-releases/cuda101/jaxlib-0.1.55-cp36-none-manylinux2010_x86_64.whl
!pip install -q -U jax
!pip install -q -U trax

     |████████████████████████████████| 144.8MB 20kB/s 
     |████████████████████████████████| 481kB 8.9MB/s 
     |████████████████████████████████| 419kB 9.7MB/s 
     |████████████████████████████████| 174kB 23.2MB/s 
     |████████████████████████████████| 1.5MB 24.2MB/s 
     |████████████████████████████████| 2.6MB 62.0MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 1.1MB 65.3MB/s 
     |████████████████████████████████| 348kB 60.8MB/s 
     |████████████████████████████████| 3.6MB 64.3MB/s 
     |████████████████████████████████| 1.1MB 63.8MB/s 
     |████████████████████████████████| 194kB 55.0MB/s 
     |████████████████████████████████| 358kB 57.4MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 5.3MB 51.2MB/s 
     |████████████████████████████████| 655kB 62.2MB/s 
     |████████████████████████████████| 983kB 54.7MB/s 
     |████████████████████████████████| 368kB 46.6MB/

In [ ]:
!pip list | grep trax

trax                          1.3.5                


In [ ]:
# clone to get vocabs and the pretrained model parts from repository
!git clone https://github.com/rickqiu/trax_chatbots.git

Cloning into 'trax_chatbots'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 81 (delta 38), reused 0 (delta 0), pack-reused 17
Unpacking objects: 100% (81/81), done.
Checking out files: 100% (11/11), done.


In [ ]:
%cd trax_chatbots
!tar -xzvf vocabs.tar.gz
!cat model_splits/* > chatbot_model1.pkl.gz
%cd ..

/content/trax_chatbots
vocabs/
vocabs/en_32k.sentencepiece
vocabs/en_32k.sentencepiece.vocab
vocabs/en_32k.subword
vocabs/en_8k.subword
/content


In [ ]:
# import libraries
import numpy as np
import trax   
from trax import layers as tl

## 2. Modelling

In [ ]:
# define attention for fast inference
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 512
    kwargs['predict_drop_len'] = 128
    return tl.SelfAttention(*args, **kwargs)

# define the model
model = trax.models.reformer.ReformerLM( 
        vocab_size=33000,
        n_layers=6,
        mode='predict',
        attention_type=attention
    )

In [ ]:
# display the Reformer model
print(str(model))

Serial[
  ShiftRight(1)
  Embedding_33000_512
  Dropout
  PositionalEncoding
  Dup_out2
  ReversibleSerial_in2_out2[
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        FastGelu
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        FastGelu
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        

## 3.  Prediction

In [ ]:
# define an input signature
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

# initialize the model from file
model.init_from_file('trax_chatbots/chatbot_model1.pkl.gz',
                     weights_only=True, input_signature=shape11)

# save the starting state
STARTING_STATE = model.state

In [ ]:
# https://www.deeplearning.ai/natural-language-processing-specialization/
# vocabulary file directory
VOCAB_DIR = './trax_chatbots/vocabs'

# vocabulary filename
VOCAB_FILE = 'en_32k.subword'

def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]


def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)


def generate_output(model, start_sentence, vocab_file, vocab_dir, temperature):
    """
    Args:
        model:  the Reformer language model you just trained
        start_sentence (string): starting sentence of the conversation
        vocab_file (string): vocabulary filename
        vocab_dir (string): directory of the vocabulary file
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """
    
    input_tokens =  tokenize(start_sentence, vocab_file, vocab_dir)
    
    # add batch dimension to array
    input_tokens_with_batch = np.expand_dims(input_tokens, axis=0)
    
    # call the autoregressive_sample_stream function from trax
    output_gen = trax.supervised.decoding.autoregressive_sample_stream( 
        model,
        inputs=input_tokens_with_batch,
        temperature=temperature
    )
    
    return output_gen


def generate_dialogue(model, model_state, start_sentence, vocab_file, vocab_dir, max_len, temperature):
    """
    Args:
        model:  the Reformer language model you just trained
        model_state (np.array): initial state of the model before decoding
        start_sentence (string): starting sentence of the conversation
        vocab_file (string): vocabulary filename
        vocab_dir (string): directory of the vocabulary file
        max_len (int): maximum number of tokens to generate 
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """  
    
    # define the delimiters we used during training
    delimiter_1 = 'Person 1: ' 
    delimiter_2 = 'Person 2: '
    
    # initialize detokenized output
    sentence = ''
    
    # token counter
    counter = 0

    # turns
    turns = 0
    
    # output tokens
    result = []
    
    # reset the model state when starting a new dialogue
    model.state = model_state
    
    # calls the output generator implemented earlier
    output = generate_output(model, start_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=temperature)
    
    # print the starting sentence
    print(start_sentence.split(delimiter_2)[0].strip())
    
    # loop below yields the next tokens until max_len is reached. the if-elif is just for prettifying the output.
    for o in output:
        
        result.append(o)
        
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)
        
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(f'{delimiter_2}{sentence}')
            sentence = ''
            result.clear()
            turns += 1
        
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(f'{delimiter_1}{sentence}')
            sentence = ''
            result.clear()

        counter += 1
        
        if counter > max_len:
           break    

## 4. Outputing results

In [ ]:
sample_sentence = ' Person 1: Is there a hospital nearby? Person 2: '
generate_dialogue(model, model_state=STARTING_STATE, 
                  start_sentence=sample_sentence,
                  vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, 
                  max_len=100, temperature=0.1)

Person 1: Is there a hospital nearby?
Person 2: Addensbrookes Hospital is located at Hills Rd, Cambridge, postcode CB20QQ. Do you need a particular department? 
Person 1: No, I just need the phone number, please. 
Person 2: The phone number is 01223245151. 
Person 1: Thank you. That's all I need. 
Person 2: Thank you for using our services.Goodbye.
Person 1: Goodbye. 


In [ ]:
sample_sentence = ' Person 1: Can you book a taxi? Person 2: '
generate_dialogue(model, model_state=STARTING_STATE, 
                  start_sentence=sample_sentence, 
                  vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, 
                  max_len=120, temperature=0.1)

Person 1: Can you book a taxi?
Person 2: I sure can. When would you like to leave? 
Person 1: I need to leave after 21:00. 
Person 2: I have booked you a taxi. The car is a yellow ford and the contact number is 07827521. 
Person 1: Thank you so much. That's all I need. 
Person 2: You're welcome. Have a nice day.
Person 1: Thank you. Goodbye. 
Person 2: You're welcome. Have a nice day.


In [ ]:
sample_sentence = ' Person 1: Are there theatres in town? Person 2: '
generate_dialogue(model, model_state=STARTING_STATE, 
                  start_sentence=sample_sentence, 
                  vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, 
                  max_len=120, temperature=0.1)

Person 1: Are there theatres in town?
Person 2: There are 4 theatres in town. Do you have a preference on area? 
Person 1: No, I don't care. Which one would you recommend? 
Person 2: I would recommend the Mumford Theatre. Would you like more information on it? 
Person 1: Yes, could I get the postcode and phone number? 
Person 2: The phone number is 08451962320 and the postcode is cb11pt. 
Person 1: Thanks. I also need a train to Cambridge on Wednesday. 
